## Code to find intersections of ICESat-2 and ATM data

**by Allison Chartrand**

**June 2019 ICESat-2 Hackweek**


In [1]:
#IMPORT PACKAGES
import os
import glob
import pandas as pd
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import pyproj
from scipy.spatial import distance

Code to extract single tracks of IceSat-2 data

In [38]:
ATL06data = pd.read_csv('../data_prod/IceSat2ZachData',parse_dates=[4])

In [3]:
ATL06data['track'].unique()

array(['gt3r', 'gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l'], dtype=object)

In [39]:
ATL06data.date.unique()

array(['2019-01-04T12:24:25.000000000', '2019-02-15T10:09:55.000000000',
       '2018-10-29T05:05:08.000000000', '2019-01-16T01:09:49.000000000',
       '2018-11-23T03:49:31.000000000', '2018-12-10T02:50:30.000000000',
       '2018-12-19T12:57:40.000000000', '2019-01-08T01:26:37.000000000',
       '2018-11-20T14:21:44.000000000', '2018-12-26T02:17:13.000000000',
       '2019-02-17T23:37:46.000000000', '2018-11-16T14:30:09.000000000',
       '2018-11-27T03:41:06.000000000', '2018-12-27T12:41:01.000000000',
       '2018-10-21T05:21:45.000000000', '2018-12-23T12:49:22.000000000',
       '2018-10-22T15:45:34.000000000', '2018-12-14T02:42:09.000000000',
       '2018-12-15T13:05:59.000000000', '2019-01-20T01:01:33.000000000',
       '2018-12-18T02:33:52.000000000', '2019-01-13T11:42:01.000000000',
       '2019-01-29T11:08:55.000000000', '2018-11-15T04:06:22.000000000',
       '2018-11-19T03:57:57.000000000', '2018-12-02T13:56:37.000000000',
       '2018-10-17T05:30:05.000000000', '2019-01-17

Define nearest neighbor algorithm

In [6]:
def closest_node(node, nodes):
    closest_index = distance.cdist([node], nodes).argmin()
    closest_dist = np.min(distance.cdist([node], nodes,'euclidean'))
    return closest_index, closest_dist

Get target points from ATM csv and put into dataframe

In [131]:
ATMdata = pd.read_csv('~/xtrak/data_prod/ATMprof_20140429.csv')

tpoints = ATMdata[['PS_x','PS_y']].copy()
tpoints = tpoints.values
ATMdata.head()

,ATM_lat,ATM_long,PS_x,PS_y,ATM_elev,dist_along
0,79.202572,341.714609,527314.046845,-1.047782e+06,32.5185,0.000000
1,79.202270,341.714622,527329.117517,-1.047811e+06,33.1235,32.996374
2,79.201968,341.714638,527344.243074,-1.047840e+06,32.7155,65.993317
3,79.201666,341.714654,527359.368654,-1.047870e+06,32.1009,98.990275
4,79.201364,341.714670,527374.494258,-1.047899e+06,30.7951,131.987250


Make an empty dataframe for intersections
Get query points from ATL06 csv and put into dataframe
Loop through unique dates and ground tracks and find the closest point between each ground track and the flowline
Store intersection along-flow distance, z_ATM, z_ATL06 and other helpful data in Intersections dataframe

In [132]:
Intersections = {'dist_along':[],'ATM_elev':[],'idx_ATM':[],'z_ATL06':[],'t_ATL06':[],'idx_ATL06':[],'gt_ATL06':[]}
Intersections = pd.DataFrame(data = Intersections)
i = 0
for day in ATL06data.date.unique():

    for tr in ATL06data.track.unique():
        close_idx = []
        min_dist = []
        AddDatarow = []

        #dfTran is a single radar transect
        dfTran = ATL06data.query('date == @day & track == @tr')
        if dfTran.shape[0] == 0:
            continue
            
        
        qpoints = dfTran[['x','y']].copy()
        qpoints = qpoints.values

        
        for j in range(len(tpoints)):
            close_idxT,min_distT = closest_node(tpoints[j,:], qpoints)
            close_idx = np.append(close_idx,[close_idxT])
            min_dist = np.append(min_dist,[min_distT])
        if min(min_dist) > 1000:
            continue
        
        tpt_idx = np.argmin(min_dist)
        qpt_idx = int(close_idx[tpt_idx])
        dfTran_idx = dfTran.loc[dfTran['date'] == day].index[0]
        qpt_idx = qpt_idx+dfTran_idx
        
        AddDatarow = {'dist_along':[ATMdata.loc[tpt_idx,'dist_along']],
                      'ATM_elev':[ATMdata.loc[tpt_idx,'ATM_elev']],
                      'idx_ATM':[tpt_idx],
                      'z_ATL06':[dfTran.loc[qpt_idx,'h']],
                      't_ATL06':[dfTran.loc[qpt_idx,'date']],
                      'idx_ATL06':[qpt_idx],
                      'gt_ATL06':[dfTran.loc[qpt_idx,'track']]}
        AddDatarow = pd.DataFrame(data = AddDatarow)
        Intersections = Intersections.append(AddDatarow,ignore_index=True)
#         Intersections.reset_index(drop=True)
        

In [75]:
Intersections.head()


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,125293.549537,36.6189,3399.0,35.714657,2019-02-15 10:09:55,1578.0,gt1l
1,125388.182985,35.0684,3402.0,34.106010,2019-02-15 10:09:55,4933.0,gt1r
2,96217.510032,88.3644,2580.0,66.844490,2019-01-16 01:09:49,39471.0,gt3r
3,102715.054408,30.4860,2761.0,29.333950,2019-01-16 01:09:49,19993.0,gt1l
4,102652.754159,33.3447,2759.0,38.831944,2019-01-16 01:09:49,23915.0,gt1r


In [16]:
%matplotlib widget

In [100]:

f, ax = plt.subplots()
plt.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],c=Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'],Intersections['t_ATL06'])
# ax.scatter(Intersections['dist_along']/1000,Intersections['z_ATL06'], c = Time,s=1)
plt.plot(Intersections['dist_along']/1000,Intersections['ATM_elev'],'.')
# cb = f.colorbar()
plt.colorbar()

# cb.ax.set_yticklabels(df.index.strftime('%b %Y'))

FigureCanvasNbAgg()

In [97]:
plt.close('all')

Save to file

In [133]:

Intersections.to_csv('~/xtrak/data_prod/Intersections_ATM20140429.csv',index=False)

In [109]:
IntersectionsSort = Intersections.sort_values(by=['t_ATL06', 'gt_ATL06'])
IntersectionsSort.reset_index()

,index,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06
0,160,122622.959615,44.7680,3314.0,43.139470,2018-10-18 15:53:52,617767.0,gt1l
1,161,122685.347575,42.9764,3316.0,42.044100,2018-10-18 15:53:52,621070.0,gt1r
2,162,125829.732109,36.7893,3416.0,36.352860,2018-10-18 15:53:52,624014.0,gt2l
3,163,125892.855741,33.7749,3418.0,32.122726,2018-10-18 15:53:52,626689.0,gt2r
4,51,80616.609191,275.4203,2136.0,262.579130,2018-10-21 05:21:45,205271.0,gt2r
5,50,77393.279691,251.8895,2036.0,247.187770,2018-10-21 05:21:45,205803.0,gt3r
6,102,67676.282654,393.7867,1775.0,358.969240,2018-10-25 05:13:27,398282.0,gt1l
7,103,67709.968188,390.8599,1776.0,389.156950,2018-10-25 05:13:27,400495.0,gt1r
8,104,64585.038715,408.5852,1708.0,2154.631000,2018-10-25 05:13:27,401953.0,gt2l
9,105,64513.127274,411.2010,1706.0,410.796300,2018-10-25 05:13:27,403761.0,gt2r


In [142]:
InterX = IntersectionsSort.values
InterX = InterX

Dist = np.diff(InterX[:,0])
Dist[Dist == 0] = float('NaN')
InterXslope = np.diff(InterX[:,3])/Dist
# i = 0
# for i in Dist():
InterXslope[np.abs(Dist) > 100] = float('NaN')
InterXslope.append(float('NaN'))
print(Dist[Dist==0])
print(InterX.shape)
print(InterXslope.shape)
print(InterXslope[0:10])

AttributeError: 'numpy.ndarray' object has no attribute 'append'